In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [2]:
key_path = key_path
project = project_id

# Sources
prices_table = 'silver_itm_prices'
prices_dataset = 'silver'
candles_table = 'silver_itm_candles'
candles_dataset = 'silver'
technical_indicators_table = 'silver_itm_technical_indicators'
technical_indicators_dataset = 'silver'
cluster_table = 'gold_clustering_sp500'
cluster_dataset = 'gold'
ticker_info_table = 'silver_ticker_info'
ticker_info_dataset = 'silver'
currency_info_table = 'silver_itm_currency_data'
currency_info_dataset = 'silver'
macro_data_table = 'silver_itm_fred_macro_data'
macro_data_dataset = 'silver'

# Tables id
table_conca_prices = f'{project}.{prices_dataset}.{prices_table}'
table_conca_candles = f'{project}.{candles_dataset}.{candles_table}'
table_conca_technical_indicators = f'{project}.{technical_indicators_dataset}.{technical_indicators_table}'
table_conca_currency = f'{project}.{currency_info_dataset}.{currency_info_table}'
table_conca_macro_data = f'{project}.{macro_data_dataset}.{macro_data_table}'
table_conca_ticker_info = f'{project}.{ticker_info_dataset}.{ticker_info_table}'
table_conca_cluster = f'{project}.{cluster_dataset}.{cluster_table}'

# Loading tables
table_to_save = 'gold_fact_data_sp500'
dataset_to_save = 'gold'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [13]:
df = bigquery.run_query(
    f"""
    SELECT
        pr.*,
        ca.candles_type,
        ca.is_candle_type,
        ti.technical_indicators,
        ti.technical_indicators_value,
        cl.cluster,
--        info.industry,
--        info.sector,
--        cu.currency,
--        cu.currency_value,
--        mc.macro_indicator,
--        mc.macro_indicator_value
    FROM {table_conca_prices} AS pr
    INNER JOIN {table_conca_candles} AS ca
        ON pr.ticker = ca.ticker AND pr.date = ca.date
--    INNER JOIN {table_conca_technical_indicators} AS ti
--        ON pr.id = ti.id  
--    INNER JOIN {table_conca_cluster} AS cl
--        ON pr.ticker = cl.ticker 
--    INNER JOIN {table_conca_ticker_info} AS info
--        ON pr.ticker = info.ticker    
--    INNER JOIN {table_conca_currency} AS cu
--        ON pr.date = cu.Date
--    INNER JOIN {table_conca_macro_data} AS mc
--        ON pr.date = mc.date
    """
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7115016 entries, 0 to 7115015
Data columns (total 11 columns):
 #   Column          Dtype              
---  ------          -----              
 0   id              object             
 1   date            datetime64[us, UTC]
 2   ticker          object             
 3   open            float64            
 4   low             float64            
 5   high            float64            
 6   close           float64            
 7   adjclose        float64            
 8   volume          Int64              
 9   candles_type    object             
 10  is_candle_type  Int64              
dtypes: Int64(2), datetime64[us, UTC](1), float64(5), object(3)
memory usage: 610.7+ MB


In [14]:
df

,id,date,ticker,open,low,high,close,adjclose,volume,candles_type,is_candle_type
0,0000225a69b6be8dd51d5702072c97f9,2020-09-08 00:00:00+00:00,PYPL,184.699997,184.699997,191.750000,185.949997,185.949997,9282400,CDL_MORNING_STAR,0
1,0000225a69b6be8dd51d5702072c97f9,2020-09-08 00:00:00+00:00,PYPL,184.699997,184.699997,191.750000,185.949997,185.949997,9282400,CDL_LONGLINE,0
2,0000225a69b6be8dd51d5702072c97f9,2020-09-08 00:00:00+00:00,PYPL,184.699997,184.699997,191.750000,185.949997,185.949997,9282400,CDL_ENGULFING,0
3,0000225a69b6be8dd51d5702072c97f9,2020-09-08 00:00:00+00:00,PYPL,184.699997,184.699997,191.750000,185.949997,185.949997,9282400,CDL_SHOOTING_STAR,0
4,0000225a69b6be8dd51d5702072c97f9,2020-09-08 00:00:00+00:00,PYPL,184.699997,184.699997,191.750000,185.949997,185.949997,9282400,CDL_DOJI,0
...,...,...,...,...,...,...,...,...,...,...,...
7115011,ffffc952bf3bb8b516c74f949593e695,2015-08-31 00:00:00+00:00,FMC,36.704250,36.019081,37.059845,36.695576,33.218033,1307733,CDL_HAMMER,0
7115012,ffffc952bf3bb8b516c74f949593e695,2015-08-31 00:00:00+00:00,FMC,36.704250,36.019081,37.059845,36.695576,33.218033,1307733,CDL_SHOOTING_STAR,0
7115013,ffffc952bf3bb8b516c74f949593e695,2015-08-31 00:00:00+00:00,FMC,36.704250,36.019081,37.059845,36.695576,33.218033,1307733,CDL_DOJI,100
7115014,ffffc952bf3bb8b516c74f949593e695,2015-08-31 00:00:00+00:00,FMC,36.704250,36.019081,37.059845,36.695576,33.218033,1307733,CDL_MORNING_STAR,0


In [ ]:
# Calculate the number of null values per column
nulls_by_column = df.isnull().sum()

# Sort the result in descending order
nulls_by_column_sorted = nulls_by_column.sort_values(ascending=False)

print(nulls_by_column_sorted)


In [17]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
